# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from requests.utils import requote_uri

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_df = pd.read_csv("../output_data/cities.csv")
cities_df.head()

,City,Lat,Lng,Max Temp(F),Humidity,Cloudiness,Wind Speed,Country,Date
0,Puerto Ayora,-0.7393,-90.3518,77.00,94,69,6.62,EC,1612674637
1,Vaini,-21.2000,-175.2000,80.60,89,75,10.36,TO,1612674927
2,Grottammare,42.9888,13.8649,48.00,94,93,5.12,IT,1612674927
3,Helena,46.5927,-112.0361,1.99,71,90,12.66,US,1612674646
4,New Norfolk,-42.7826,147.0587,72.00,45,15,4.00,AU,1612674745


In [3]:
cities_df.shape

(588, 9)

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [13]:
#Lets handle Null values
cities_df = cities_df.dropna()

#Fetching location
locations = cities_df[["Lat","Lng"]].astype(float)

#Fetch humidity for weight
Humidity = cities_df["Humidity"].astype(float)


In [6]:
# Create a poverty Heatmap layer
layout = {
    'width': '960px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '3px',
    'margin': '0 auto 0 auto'}
fig = gmaps.figure(layout=layout,zoom_level=2.15,center=(19,23))

heat_layer = gmaps.heatmap_layer(locations,
                                 weights=Humidity,
                                 dissipating=False,
                                 max_intensity=60,
                                 point_radius = 2)
fig.add_layer(heat_layer)

fig


Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='3px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
Ideal_df = cities_df[(cities_df["Max Temp(F)"]<80)
                     &(cities_df["Max Temp(F)"]>70) 
                     & (cities_df["Wind Speed"]<10)
                     & (cities_df["Cloudiness"] == 0)]
Ideal_df.reset_index(drop=True)

,City,Lat,Lng,Max Temp(F),Humidity,Cloudiness,Wind Speed,Country,Date
0,Hong Kong,22.2855,114.1577,73.99,58,0,1.01,HK,1612674956
1,Morón,-34.6534,-58.6198,73.99,92,0,3.00,AR,1612674982
2,Nāndūra Buzurg,20.8333,76.4500,74.68,22,0,7.11,IN,1612675003
3,Puerto Escondido,15.8500,-97.0667,76.53,75,0,5.30,MX,1612675005
4,Brejo Santo,-7.4933,-38.9872,78.80,57,0,4.61,BR,1612675006
5,Acapulco de Juárez,16.8634,-99.8901,74.82,77,0,3.42,MX,1612675010
6,Acopiara,-6.0953,-39.4525,70.81,84,0,4.47,BR,1612675015
7,Umarkot,25.3616,69.7362,75.07,18,0,7.40,PK,1612675025
8,Kati,12.7441,-8.0726,75.20,25,0,8.05,ML,1612675044


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
hotel_df = Ideal_df.reset_index(drop = True)

#Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] =""
hotel_df.head(3)

,City,Lat,Lng,Max Temp(F),Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Hong Kong,22.2855,114.1577,73.99,58,0,1.01,HK,1612674956,
1,Morón,-34.6534,-58.6198,73.99,92,0,3.00,AR,1612674982,
2,Nāndūra Buzurg,20.8333,76.4500,74.68,22,0,7.11,IN,1612675003,


In [9]:
#Set parameters to search for hotels with 5000 meters.
# geocoordinates
# set up a parameters dictionary
params = {
    "radius": 50000,
    "types": "lodging",
    "keyword": "Hotel",
    "key": g_key
}

# Use the lat/lng we recovered to identify airports
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Set the base URl
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params).json()
    
    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")


In [10]:
hotel_df

,City,Lat,Lng,Max Temp(F),Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Hong Kong,22.2855,114.1577,73.99,58,0,1.01,HK,1612674956,K11 ARTUS
1,Morón,-34.6534,-58.6198,73.99,92,0,3.00,AR,1612674982,Duque Hotel Boutique & Spa
2,Nāndūra Buzurg,20.8333,76.4500,74.68,22,0,7.11,IN,1612675003,HOTEL GRAND SUN
3,Puerto Escondido,15.8500,-97.0667,76.53,75,0,5.30,MX,1612675005,Selina Puerto Escondido
4,Brejo Santo,-7.4933,-38.9872,78.80,57,0,4.61,BR,1612675006,Hotel Ibis
5,Acapulco de Juárez,16.8634,-99.8901,74.82,77,0,3.42,MX,1612675010,Banyan Tree Cabo Marqués
6,Acopiara,-6.0953,-39.4525,70.81,84,0,4.47,BR,1612675015,ELLO HOTEL
7,Umarkot,25.3616,69.7362,75.07,18,0,7.40,PK,1612675025,Ghosia Guest House
8,Kati,12.7441,-8.0726,75.20,25,0,8.05,ML,1612675044,Hôtel l’Amitié Bamako


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [12]:
# Add marker layer ontop of heat map
# Create a map using City coordinates to set markers
# Create a marker_layer using the name , city and country to fill the info box
fig = gmaps.figure(layout=layout,zoom_level=2.15,center=(19,23))
markers = gmaps.marker_layer(locations,info_box_content=hotel_info)

#add heat layer
fig.add_layer(heat_layer)

#Add merkers
fig.add_layer(markers)

# Display figure
fig


Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='3px', wi…